In [ ]:
import os
import torch
import torchvision as tv
import torchio as tio
import random
import pandas as pd
import math
import csv

images_dir = "/home/fengling/Documents/prl/data/processed"
subjects_df = pd.read_csv("/home/fengling/Documents/prl/prl_pytorch/data/cv_df.csv")

if os.getenv("LSB_JOBINDEX") == None:
    i = 1
else:
    i = int(os.getenv("LSB_JOBINDEX"))

train = subjects_df["subject_id"][subjects_df["cv_index"] != i - 1].values.tolist()
test = subjects_df["subject_id"][subjects_df["cv_index"] == i - 1].values.tolist()

      
# Data loader settings
transforms = [
    tio.RandomAffine(
        scales=(0.66, 1.5), 
        degrees=180,
        isotropic=True
    ),
    tio.RandomFlip()
]
transform = tio.Compose(transforms)

patch_size = 24
queue_length = 500
samples_per_volume = 50
num_workers = 20
sampler = tio.data.WeightedSampler(patch_size, "prob")

# Train set data loader
train_dir = [os.path.join(images_dir, x) for x in train]

train_subject_list = []
for subject_path in train_dir:
    file_paths = [os.path.join(subject_path, x) for x in os.listdir(subject_path)]
    subject = tio.Subject(
        epi = tio.ScalarImage([s for s in file_paths if "epi_final.nii.gz" in s][0]),
        flair = tio.ScalarImage([s for s in file_paths if "flair_final.nii.gz" in s][0]),
        phase = tio.ScalarImage([s for s in file_paths if "phase_final.nii.gz" in s][0]),
        t1 = tio.ScalarImage([s for s in file_paths if "t1_final.nii.gz" in s][0]),
        prob = tio.ScalarImage([s for s in file_paths if "prob.nii.gz" in s][0]),
        lesion_mask = tio.LabelMap([s for s in file_paths if "prob_01.nii.gz" in s][0]),
        lesion_type = tio.LabelMap([s for s in file_paths if "lesion_labels.nii.gz" in s][0]),
        name = subject_path
    )
    train_subject_list.append(subject)

train_dataset = tio.SubjectsDataset(train_subject_list, transform = transform)

train_queue = tio.Queue(
    train_dataset,
    queue_length,
    samples_per_volume,
    sampler,
    num_workers=num_workers
)

train_loader = torch.utils.data.DataLoader(
    train_queue,
    batch_size=64,
    num_workers=0,  # this must be 0
)

# Test set data loader
test_dir = [os.path.join(images_dir, x) for x in test]

test_subject_list = []
for subject_path in test_dir:
    file_paths = [os.path.join(subject_path, x) for x in os.listdir(subject_path)]
    subject = tio.Subject(
        epi = tio.ScalarImage([s for s in file_paths if "epi_final.nii.gz" in s][0]),
        flair = tio.ScalarImage([s for s in file_paths if "flair_final.nii.gz" in s][0]),
        phase = tio.ScalarImage([s for s in file_paths if "phase_final.nii.gz" in s][0]),
        t1 = tio.ScalarImage([s for s in file_paths if "t1_final.nii.gz" in s][0]),
        prob = tio.ScalarImage([s for s in file_paths if "prob.nii.gz" in s][0]),
        lesion_mask = tio.LabelMap([s for s in file_paths if "prob_01.nii.gz" in s][0]),
        lesion_type = tio.LabelMap([s for s in file_paths if "lesion_labels.nii.gz" in s][0]),
        name = subject_path
    )
    test_subject_list.append(subject)

test_dataset = tio.SubjectsDataset(test_subject_list)

test_queue = tio.Queue(
    test,
    queue_length,
    samples_per_volume,
    sampler,
    num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    test_queue,
    batch_size=64,
    num_workers=0,  # this must be 0
)

In [24]:
i = 1

In [25]:
"hello" + str(i) + ".pt"

'hello1.pt'